# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

In [2]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        '''ToDo: fix this function'''
        return self.w / self.C

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        '''ToDo: fix this function'''
        if 1 - answer * self.f(x) > 0:
            return -1.0 
        else:
            return 0.0

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k

            # w update
            '''ToDo: add w update with regularization'''  
            self.w -= (self.der_loss(x, y) + self.der_reg()) * step
            
            # w_0 update
            '''ToDo: add w_0 update'''
            self.w0 -= self.der_loss(x, y) * step

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [3]:
model = MySVM()
model.fit(X_train, y_train)
print(model.w, model.w0)

[ 4.40506966  4.98095583  4.7623448   4.62907599  4.18665741  4.18663331
  4.08877134  4.89645992  4.63153129  4.73843539  4.05129097  5.00014178
  4.86274328  4.24294973  4.21245084  4.21402959  4.33480692  4.55521088
  4.46244586  4.32180032] 2.5189384745422867


In [4]:
predictions = model.predict(X_test)

In [5]:
print(predictions)

[0 0 1 ..., 1 1 0]


In [6]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ..., 1 0 1] 2000 991


In [7]:
print(len(predictions), sum(predictions))

2000 981


In [8]:
print(sum(predictions == y_test) / float(len(y_test)))

0.544


## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

In [14]:
from sklearn.svm import LinearSVC

In [15]:
svc = LinearSVC(penalty='l2', loss='hinge', C=10000, max_iter=10000)
svc.fit(X_train, y_train)
preds = svc.predict(X_test)

In [16]:
from sklearn.metrics import accuracy_score

print(accuracy_score(preds, y_test))

0.7
